In [21]:
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
import sys

In [4]:
%%time
# Выгружаем из xlsx файла

df = pd.read_excel('../data/temp_fact.xlsx')
df

CPU times: user 3.7 s, sys: 60.5 ms, total: 3.76 s
Wall time: 3.78 s


,vm,metric,value,timestamp
0,DataLake-DBN1,cpu.usage.average,5.110000e+00,2025-11-25 17:00:00
1,DataLake-DBN1,cpu.usagemhz.average,2.450000e+03,2025-11-25 17:00:00
2,DataLake-DBN1,cpu.ready.summation,7.925000e+04,2025-11-25 17:00:00
3,DataLake-DBN1,mem.consumed.average,1.338407e+08,2025-11-25 17:00:00
4,DataLake-DBN1,mem.overhead.average,5.006010e+05,2025-11-25 17:00:00
...,...,...,...,...
96894,VSA-Lenovo1,disk.unshared.latest,1.613255e+11,2025-12-01 23:30:00
96895,VSA-Lenovo1,disk.usage.average,2.476680e+05,2025-12-01 23:30:00
96896,VSA-Lenovo1,disk.used.latest,1.613262e+11,2025-12-01 23:30:00
96897,VSA-Lenovo1,net.usage.average,3.809400e+04,2025-12-01 23:30:00


In [9]:
metric = 'cpu.usage.average'
vm = 'DataLake-DBN1'

df = df[df['vm']==vm]
df = df[df['metric']==metric]

df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %h:%m:%s")
df.sort_values(by=['timestamp'], inplace=True)
df = df.rename(columns={'timestamp':'ds', 'value': 'y'})
df

,vm,metric,y,ds
0,DataLake-DBN1,cpu.usage.average,5.11,2025-11-25 17:00:00
16,DataLake-DBN1,cpu.usage.average,5.03,2025-11-25 17:30:00
32,DataLake-DBN1,cpu.usage.average,4.77,2025-11-25 18:00:00
48,DataLake-DBN1,cpu.usage.average,4.68,2025-11-25 18:30:00
64,DataLake-DBN1,cpu.usage.average,4.63,2025-11-25 19:00:00
...,...,...,...,...
4752,DataLake-DBN1,cpu.usage.average,5.38,2025-12-01 21:30:00
4768,DataLake-DBN1,cpu.usage.average,5.08,2025-12-01 22:00:00
4784,DataLake-DBN1,cpu.usage.average,5.13,2025-12-01 22:30:00
4800,DataLake-DBN1,cpu.usage.average,5.11,2025-12-01 23:00:00


In [13]:
# обобщить нормально TODO
df_train = df.head(-48)
df_train = df_train.drop(columns=['vm', 'metric'])
df_train.to_excel('../data/DataLake-DBN1_cpu-usage_train.xlsx')

In [14]:
df_test = df.tail(48)
df_test = df_test.drop(columns=['vm', 'metric'])
df_test.to_excel('../data/DataLake-DBN1_cpu-usage_test.xlsx')

In [22]:
# Выгружаем из базы данных
# Конфигурация
DB_CONFIG = {
    'host': 'localhost',
    'port': '5432',
    'database': 'server_metrics',
    'user': 'postgres',
    'password': 'postgres'
}

In [23]:
def db_export(vm, metric, start_date, end_date):
    """
    Выгрузка данных за период времени

    Args:
        vm: сервер
        metric: метрика
        start_date: выгрузка С
        end_date: выгрузка ПО
    """

    try:
        # Подключение к базе
        conn = psycopg2.connect(**DB_CONFIG)

        # SQL запрос
        query = """
        SELECT
            timestamp,
            value
        FROM server_metrics_fact
        WHERE
            vm = %s
            AND metric = %s
            AND timestamp BETWEEN %s AND %s
        ORDER BY timestamp ASC
        """

        # Выполнение запроса
        df = pd.read_sql_query(query, conn, params=(vm, metric, start_date, end_date))

        # Закрытие соединения
        conn.close()

        if df.empty:
            print(f"Данные не найдены для {vm} - {metric}")
            return None

        # Вывод информации
        print(f"Выгружено {len(df)} записей")
        print(f"Период: {df['timestamp'].min()} - {df['timestamp'].max()}")
        print(f"Значения: {df['value'].min():.2f}% - {df['value'].max():.2f}%")
        print(f"Среднее: {df['value'].mean():.2f}%")

        return df

    except Exception as e:
        print(f"Ошибка: {e}")
        return None

In [24]:
# Пример использования
data = db_export("DataLake-DBN1", "cpu.usage.average", '2025-11-26 01:00:00', '2025-11-28 01:00:00')
data

Выгружено 97 записей
Период: 2025-11-26 01:00:00+00:00 - 2025-11-28 01:00:00+00:00
Значения: 4.24% - 5.08%
Среднее: 4.70%


/var/folders/n6/9hwnx40x1g53w79wdp1sqz5w0000gn/T/ipykernel_23790/207106136.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(vm, metric, start_date, end_date))


,timestamp,value
0,2025-11-26 01:00:00+00:00,4.63
1,2025-11-26 01:30:00+00:00,4.37
2,2025-11-26 02:00:00+00:00,4.42
3,2025-11-26 02:30:00+00:00,4.43
4,2025-11-26 03:00:00+00:00,4.58
...,...,...
92,2025-11-27 23:00:00+00:00,4.73
93,2025-11-27 23:30:00+00:00,4.87
94,2025-11-28 00:00:00+00:00,4.81
95,2025-11-28 00:30:00+00:00,4.77
